In [1]:
# Essential libraries
import numpy as np 
import pandas as pd
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
from tensorflow.keras import applications,activations
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras import optimizers,utils
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization,ZeroPadding2D, Input
from tensorflow.keras.layers import Conv2D, Activation,MaxPooling2D
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,SeparableConv2D,MaxPooling2D,BatchNormalization,SpatialDropout2D,Dropout,Activation,Dense,Lambda,Input,Flatten
from tensorflow.keras.regularizers import l2

import tensorflow as tf

def make_default_hidden_layers(inputs):

    x = SeparableConv2D(32, (3, 3), padding="same")(inputs)
    x = BatchNormalization(axis=-1)(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)

    x = SeparableConv2D(64, (3, 3), padding="same")(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = SeparableConv2D(128, (3, 3), padding="same")(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)

    x = SeparableConv2D(256, (3, 3), padding="same")(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)
    

    return x

def build_gender_branch(inputs):
  
    x = make_default_hidden_layers(inputs)
    x = Flatten()(x)
    x = Dense(2)(x)
    x = Activation("softmax", name="gender_output")(x)

    return x

def build_age_branch(inputs):   

    x = make_default_hidden_layers(inputs)
    x = Flatten()(x)
    x = Dense(5)(x)
    
## For the second model
#     x = Dense(7)(x)
    
    x = Activation("softmax", name="age_output")(x)  

    return x


def assemble_model(width, height):
  
    input_shape = (height, width, 1)
    inputs = Input(shape=input_shape)

    age_branch = build_age_branch(inputs)
    gender_branch = build_gender_branch(inputs)

    model = Model(inputs=inputs, outputs = [age_branch, gender_branch], name="face_net")
    
## For the second model
#     model = Model(inputs=inputs, outputs = age_branch, name="face_net")

    return model
    
model = assemble_model(98, 98)


In [ ]:
## For the second model
# class weight

# from sklearn.utils import class_weight
# class_weight = class_weight.compute_class_weight( class_weight='balanced' , classes=np.array([0,1,2,3,4,5,6]) , y=np.array(df['age_group']))
# class_weights = dict(enumerate(class_weight))


In [ ]:
# callback
import math
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler

#  When training a neural network, the learning rate is often the most important 
# hyperparameter to tune. When training deep neural networks, it is often useful 
# to reduce learning rate as the training progresses.

# LRS in Keras reduces the learning rate by a certain factor after certain no of epochs

def step_decay(epoch):
	initial_lrate = 0.008
	drop = 0.5
	epochs_drop = 5.0
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate      

opt = Adam(lr=0.0)                                   
lrate = LearningRateScheduler(step_decay)

model.compile(optimizer=opt, 
              loss={
                  'age_output': 'categorical_crossentropy', 
                  'gender_output': 'categorical_crossentropy'},
              metrics={
                  'age_output': 'accuracy', 
                  'gender_output': 'accuracy'})

## For the second model
# model.compile(optimizer=opt, 
#               loss={
#                   'age_output': 'categorical_crossentropy'},
#               metrics={
#                   'age_output': 'accuracy'})

callbacks_list = [lrate]


In [ ]:
train_gen = generate_images(train_idx, is_training=True, batch_size=32)
valid_gen = generate_images(val_idx, is_training=True, batch_size=32)

history = model.fit(train_gen, steps_per_epoch = len(train_idx)//32, epochs=40  , callbacks=callbacks_list,
                     validation_data=valid_gen, validation_steps=len(val_idx)//32)


## For the second model
# history = model.fit(train_gen, class_weight = class_weights, steps_per_epoch = len(train_idx)//32, epochs=25  , callbacks=callbacks_list, 
#                     validation_data=valid_gen, validation_steps=len(val_idx)//32)

model.save("./grey_age5_drop02.h5")

In [ ]:
import plotly.graph_objects as go
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter(
                    y=history.history['gender_output_accuracy'],
                    name='Train'))
fig.add_trace(go.Scatter(
                    y=history.history['val_gender_output_accuracy'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for gender feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()

In [ ]:
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scattergl(
#                     y=history.history['age_output_accuracy'],
                    y=history.history['accuracy'],
                    name='Train'))
fig.add_trace(go.Scattergl(
#                     y=history.history['val_age_output_accuracy'],
                    y=history.history['val_accuracy'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for age feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()